# NFDRS Version 4 Index Calculator
### Contact: W. Matt Jolly, william.jolly@usda.gov
##### Version: 1.0 (13 Mar 2025)
### Source: https://github.com/firelab/NFDRS4/blob/master/lib/NFDRS4/src/nfdrs4.cpp
#### Ported from the iCalcIndexes function used to calculate Spread Component, Energy Release Component, Burning Index and Ignition Component

In [1]:
from math import *
import numpy as np
import matplotlib.pyplot as plt

In [31]:
# Create the fuel model
# Fuel Loads (tons/acre)
L1 = 2.5         # 1 hr
L10 = 2.2        # 10 hr
L100 = 3.6       # 100 hr
L1000 = 10.16    # 1000 hr
LWOOD = 0.0      # Woody
LHERB = 0.0      # Herbaceous
LDROUGHT = 5.0   # Drought fuel loading

# Surface-area-to-volume ratio of fuels (ft-1)
SG1 = 2000       # 1 hr
SG10 = 109       # 10 hr
SG100 = 30       # 100 hr
SG1000 = 8       # 1000 hr
SGWOOD = 1500    # Woody
SGHERB = 2000    # Herbaceous

HD = 8000        # Fuel Heat Content (BTU)
SCM = 5          # Max Spread Component (DIM)
WNDFC = 0.2      # Wind Adjustment Factor (DIM)
DEPTH = 0.6      # Fuel Bed Depth (ft)
MXD = 25         # Dead fuel moisture of extinction (% dry wt)
CTA = 0.0459137  # Convert tons per acres to pounds per square foot

#### Define a fuel moisture scenario for use in the calculations

![NFDRS4 Calculator Example](NDFDRCalculatorExample.png)

In [32]:
MC1 = 7      # 1 hr
MC10 = 8     # 10 hr
MC100 = 9    # 100 hr
MC1000 = 20  # 1000 hr
MCWOOD = 60  # Woody
MCHERB = 60  # Herb

#### Other variables the Index Calculator needs

In [60]:
GSI = 0.5                # GSI used for Curing Function
GreenupThreshold = 0.2   # Green-up threshold used for Curing Function
GSIMax = 1.0             # Max GSI used for Curing Function
KBDI = 100               # 'Actual' KBDI value to use for testing (range 0 to 800)
KBDIThreshold = 100      # KBDI threshold defines the start of the drought fuel loading addition
WS = 5                   # Windspeed (mph)
SlopeCls = 1             # Slope Class (scalar)  Time-invariant
UseActualSlope = False   # There is an option to use the actual slope of the site (%)

In [34]:
# Convert degrees to radians
def degreesToRadians(angleDegrees):
    return (angleDegrees * 3.14159 / 180.0)


#### Model constants for the mineral content, particle density 

In [35]:
STD = STL = .0555 
RHOD = RHOL = 32
ETASD = ETASL = 0.4173969   # Dead and live fuel mineral damping coefficient

#### Convert all fuel loads from tons/acre to lbs/ft^2 (using CTA)

In [36]:
# Convert all fuel loads from tons/acre to lbs/ft^2 (using CTA)
W1 = L1 * CTA
W10 = L10 * CTA
W100 = L100 * CTA
W1000 = L1000 * CTA
WWOOD = LWOOD * CTA
WHERB = LHERB * CTA
WWOOD = LWOOD * CTA
WDROUGHT = LDROUGHT * CTA
fDEPTH = DEPTH

### Apply the drought fuel loading 
#### Based on KBDI

In [37]:
if (KBDI > KBDIThreshold ):
    WTOTD = W1 + W10 + W100
    WTOTL = WHERB + WWOOD
    WTOT = WTOTD + WTOTL
    PackingRatio = WTOT / fDEPTH
    if (PackingRatio == 0):
        PackingRatio = 1.0
    WTOTD = WTOTD + W1000
    # Calculate the 'drought unit' which is a fraction of drought fuel load applied to the fuel model
    DroughtUnit = WDROUGHT / (800.0 - KBDIThreshold)
    W1 = W1 + (W1 / WTOTD) * (KBDI - KBDIThreshold) * DroughtUnit
    W10 = W10 + (W10 / WTOTD) * (KBDI - KBDIThreshold) * DroughtUnit
    W100 = W100 + (W100 / WTOTD) * (KBDI - KBDIThreshold) * DroughtUnit
    W1000 = W1000 + (W1000 / WTOTD) * (KBDI - KBDIThreshold) * DroughtUnit
    WTOT = W1 + W10 + W100 + W1000 + WTOTL
    fDEPTH = (WTOT - W1000) / PackingRatio

### Calculate curing and adjust 1hr and herbaceous fuel loads 
#### Based on the Herbaceous GSI, GSIMax and Greenup Threshold

In [38]:
if  (GSI <= GreenupThreshold):
    # If GSI is less than Green-up, then everything is fully cured
    fctCur = 1 
else:
    # If GSI > GreenupTreshold, calculate the curing percentage
    fctCur = -1.0 /(1.0 - GreenupThreshold) * (GSI/GSIMax) + 1.0/(1.0 - GreenupThreshold)
if (fctCur < 0):
    fctCur = 0.0
if (fctCur > 1):
    fctCur = 1.0

# Adjust the 1hr and herb fuel loads for curing
W1P = W1 + WHERB * fctCur                           # 1hr Fuel Load after curing
WHERBP = WHERB * (1 - fctCur)                       # Herbaceous Fuel Load after curing

#### Calculate summary total loadings by fuel types

In [39]:
WTOTD = W1P + W10 + W100 + W1000					# Total Dead Fuel Loading
WTOTL = WHERBP + WWOOD								# Total Live Fuel Loading
WTOT = WTOTD + WTOTL								# Total Fuel Loading

# Calculate the Net Fuel Load, this basically just adjusts for the ash or inorganic content of the fuels (assumed 5.55%)
W1N = W1P * (1.0 - STD)							    # Net 1hr Fuel Loading (Note: This applies the 1hr fuel load after curing (W1P)
W10N = W10 * (1.0 - STD)							# Net 10hr Fuel Loading
W100N = W100 * (1.0 - STD)							# Net 100hr Fuel Loading
WHERBN = WHERBP * (1.0 - STL)						# Net Herbaceous Fuel Loading (Note: This applies the herb fuel load after curing (WHERBP)
WWOODN = WWOOD * (1.0 - STL)						# Net Woody Fuel Loading
WTOTLN = WTOTL * (1.0 - STL)						# Net Total Live Fuel Lodaing

# Calculate the bulk density characteristics of the fuel bed
RHOBED = (WTOT - W1000) / fDEPTH					# Bulk density of the fuel bed
RHOBAR = ((WTOTL * RHOL) + (WTOTD * RHOD)) / WTOT   # Weighted particle density of the fuel bed
BETBAR = RHOBED / RHOBAR							# Ratio of bulk density to particle density

#### If live net fuel loading is greater than 0, calculate the Live Fuel Moisture of Extinction (MXL)

In [40]:
if (WTOTLN > 0):
    HN1 = W1N * exp(-138.0 / SG1)
    HN10 = W10N  * exp(-138.0 / SG10)
    HN100 = W100N * exp(-138.0 / SG100)
    if ((-500 / SGHERB) < -180.218):
        HNHERB = 0
    else:
        HNHERB = WHERBN * exp(-500.0 / SGHERB)
    
    if ((-500 / SGWOOD) < -180.218):
        HNWOOD = 0
    else:
        HNWOOD = WWOODN * exp(-500.0 / SGWOOD)

    if ((HNHERB + HNWOOD) == 0):
        WRAT = 0
    else:
        WRAT = (HN1 + HN10 + HN100) / (HNHERB + HNWOOD)
    MCLFE = ((MC1 * HN1) + (MC10 * HN10) + (MC100 * HN100)) / (HN1 + HN10 + HN100)
    MXL = (2.9 * WRAT * (1.0 - MCLFE / MXD) - 0.226) * 100

else:
    MXL = 0

if (MXL < MXD): MXL = MXD

### Calculate the surface area of the 6 fuel classes
#### SA = (Loadings / Density) * Surface-area-to-volume ratio

##### Note: These have to be done after the curing and load transfer has taken place

In [41]:
SA1 = (W1P / RHOD) * SG1            # Surface area of dead 1hr fuel (Note: This applies the 1hr fuel load after curing (W1P)
SA10 = (W10 / RHOD) * SG10          # Surface area of dead 10hr fuel
SA100 = (W100 / RHOD) * SG100       # Surface area of dead 100hr fuel
SAHERB = (WHERBP / RHOL) * SGHERB   # Surface area of live herbaceous fuel (Note: This applies the 1hr fuel load after curing (WHERBP)
SAWOOD = (WWOOD / RHOL) * SGWOOD    # Surface area of live woody fuel
SADEAD = SA1 + SA10 + SA100		    # Surface area of dead fuel
SALIVE = SAHERB + SAWOOD			# Surface area of live fuel

In [42]:
F1 = SA1 / SADEAD      #Proportion of dead-fuel surface area in 1-hour class,used as a weighting factor for ROS calculation
F10 = SA10 / SADEAD    #Proportion of dead-fuel surface area in 10-hour class,used as a weighting factor for ROS calculation
F100 = SA100 / SADEAD  #Proportion of dead-fuel surface area in 100-hour class,used as a weighting factor for ROS calculation

In [43]:
F1 + F10 + F100

1.0

In [44]:
if (WTOTL <=0):
   FHERB = FWOOD = 0
else:
   FHERB = SAHERB / SALIVE
   FWOOD = SAWOOD / SALIVE
    
FDEAD = SADEAD / (SADEAD + SALIVE)		# Fraction of Dead Fuel Surface area to total loading
FLIVE = SALIVE / (SADEAD + SALIVE)		# Fraction of Live Fuel Surface area to total loading

WDEADN = (F1 * W1N) + (F10 * W10N) + (F100 * W100N)	# Weighted dead-fuel loading, (NOTE: This weighting ignores 1000hr fuels)

if (SGWOOD > 1200 and SGHERB > 1200):
    WLIVEN = WTOTLN
else:
    WLIVEN = (FWOOD * WWOODN) + (FHERB * WHERBN)

### Calculate the Spread Component

In [45]:
# Characteristic surface area-to-volume ratio of dead fuel, surface area weighted
SGBRD = (F1 * SG1) + (F10 * SG10) + (F100 * SG100)
# Characteristic surface area-to-volume ratio of live fuel, surface area weighted.
SGBRL = (FHERB * SGHERB) + (FWOOD * SGWOOD)
# Characteristic surface area-to-volume ratio of fuel bed, surface area weighted.
SGBRT = (FDEAD * SGBRD) + (FLIVE * SGBRL)
# Optimum packing ratio, surface area weighted
BETOP = 3.348 * pow(SGBRT, -0.8189)
# Weighted maximum reaction velocity of surface area
GMAMX = pow(SGBRT, 1.5) / (495.0 + 0.0594 * pow(SGBRT, 1.5))
AD = 133 * pow(SGBRT, -0.7913)
# Weighted optimum reaction velocity of surface area
GMAOP = GMAMX * pow((BETBAR / BETOP), AD) * exp(AD * (1.0 - (BETBAR / BETOP)))
ZETA = exp((0.792 + 0.681 * pow(SGBRT, 0.5)) * (BETBAR + 0.1))
ZETA = ZETA / (192.0 + 0.2595 * SGBRT)

In [46]:
def Limit(val,low,high):
    if (val < low):
        val = low
    if (val > high):
        val = high
    return val

In [47]:
WTMCD = (F1 * MC1) + (F10 * MC10) + (F100 * MC100)
WTMCL = (FHERB * MCHERB) + (FWOOD * MCWOOD)
DEDRT = WTMCD / MXD
LIVRT = WTMCL / MXL
# Moisture damping
ETAMD = 1.0 - 2.59 * DEDRT + 5.11 * pow(DEDRT,2.0) - 3.52 * pow(DEDRT, 3.0)
ETMAD = Limit(ETAMD,0,1)
ETAML = 1.0 - 2.59 * LIVRT + 5.11 * pow(LIVRT,2.0) - 3.52 * pow(LIVRT, 3.0)
ETMAD = Limit(ETAML,0,1)

B = 0.02526 * pow(SGBRT, 0.54)
C = 7.47 * exp(-0.133 * pow(SGBRT,0.55))
E = 0.715 * exp(-3.59 * pow(10.0, -4.0) * SGBRT)
UFACT = C * pow(BETBAR / BETOP, -1 * E)
# HL = HD */

IR = GMAOP * ((WDEADN * HD * ETASD * ETAMD) + (WLIVEN * HD * ETASL * ETAML))

In [48]:
IR  # Total Reaction Intensity

2519.449397542792

In [49]:
if (88.0 * WS * WNDFC > 0.9 * IR):
    PHIWND = UFACT * pow(0.9 * IR, B)
else:
    PHIWND = UFACT * pow(WS * 88.0 * WNDFC, B)
match SlopeCls:
    case 1:
        slpfct = 0.267
    case 2:
        slpfct = 0.533
    case 3:
        slpfct = 1.068
    case 4:
        slpfct = 2.134
    case 4:
        slpfct = 4.273
    case _:
       slpfct = 0.267
# Actual slopes in degrees (>5) can now be input
# Matches forumla used in WIMS developed by Larry Bradshaw (31 Aug 2016)
if UseActualSlope:
    slpfct = 5.275 * (tan(degreesToRadians(SlopeCls)))
    
PHISLP = slpfct * pow(BETBAR, -0.3)
XF1 = F1 * exp(-138.0 / (SG1)) * (250.0 + 11.16 * MC1)
XF10 = F10 * exp(-138.0 / (SG10)) * (250.0 + 11.16 * MC10)
XF100 = F100 * exp(-138.0 / (SG100)) * (250.0 + 11.16 * MC100)
XFHERB = FHERB * exp(-138.0 / (SGHERB)) * (250.0 + 11.16 * MCHERB)
XFWOOD = FWOOD * exp(-138.0 / (SGWOOD)) * (250.0 + 11.16 * MCWOOD)
HTSINK = RHOBED * (FDEAD * (XF1 + XF10 + XF100) + FLIVE * (XFHERB + XFWOOD))
SC = IR * ZETA * (1.0 + PHISLP + PHIWND) / HTSINK

In [50]:
SC

2.0696607886011473

### Calculate Energy Release Component and subsequent Burning Index

In [51]:
# Fraction of fuel loadings for Energy Release Component calculations
F1E = W1P / WTOTD
F10E = W10 / WTOTD
F100E = W100 / WTOTD
F1000E = W1000 / WTOTD

In [52]:
F1E + F10E + F100E + F1000E

1.0

In [53]:
# Fraction of fuel loadings for Energy Release Component calculations
F1E = W1P / WTOTD
F10E = W10 / WTOTD
F100E = W100 / WTOTD
F1000E = W1000 / WTOTD

if (WTOTL <=0):
    FHERBE = FWOODE =  0
else:
    FHERBE = WHERBP / WTOTL
    FWOODE = WWOOD / WTOTL

FDEADE = WTOTD / WTOT
FLIVEE = WTOTL / WTOT
WDEDNE = WTOTD * (1.0 - STD)
WLIVNE = WTOTL * (1.0 - STL)
SGBRDE = (F1E * SG1) + (F10E * SG10) + (F100E * SG100) + (F1000E * SG1000)
SGBRLE = (FHERBE * SGHERB) + (FWOODE * SGWOOD)
SGBRTE = (FDEADE * SGBRDE) + (FLIVEE * SGBRLE)
BETOPE = 3.348 * pow(SGBRTE, -0.8189)


In [54]:
GMAMXE = pow(SGBRTE, 1.5) / (495.0 + 0.0594 * pow(SGBRTE, 1.5))
ADE = 133 * pow(SGBRTE, -0.7913)
GMAOPE = GMAMXE * pow( (BETBAR/BETOPE), ADE) * exp(ADE * (1.0 - (BETBAR / BETOPE)))

WTMCDE = (F1E * MC1) + (F10E * MC10) + (F100E * MC100) + (F1000E * MC1000)
WTMCLE = (FHERBE * MCHERB) + (FWOODE * MCWOOD)
DEDRTE = WTMCDE / MXD
LIVRTE = WTMCLE / MXL
ETAMDE = 1.0 - 2.0 * DEDRTE + 1.5 * pow(DEDRTE,2.0) - 0.5 * pow(DEDRTE, 3.0)
ETAMLE = 1.0 - 2.0 * LIVRTE + 1.5 * pow(LIVRTE,2.0) - 0.5 * pow(LIVRTE, 3.0)
if (ETAMDE < 0): ETAMDE = 0
if (ETAMDE > 1): ETAMDE = 1
if (ETAMLE < 0): ETAMLE = 0
if (ETAMLE > 1): ETAMLE = 1

IRE = (FDEADE * WDEDNE * HD * ETASD * ETAMDE)
# HL = HD */
IRE = GMAOPE * (IRE + (FLIVEE * WLIVNE * (HD) * ETASL * ETAMLE))
TAU = 384.0 / SGBRT
ERC = 0.04 * IRE * TAU
BI = (.301 * pow((SC * ERC), 0.46)) * 10.0

In [55]:
ERC

29.16603373213386

In [56]:
BI

19.84831786828181

### Calculate Ignition Component

In [57]:
FuelTemperature = 26.6667

In [58]:
# Finally, calculate the Igntion Component
TMPPRM = 0.0
PNORM1 = 0.00232
PNORM2 = 0.99767
QIGN = 0.0
CHI = 0.0
PI = 0.0
SCN = 0.0
PFI = 0.0
IC = 0.0
if (SCM <= 0): IC = 0

# Replace iTemp with the Nelson-derived fuel surface temperature
TMPPRM = FuelTemperature
#TMPPRM = OneHourFM.meanWtdTemperature()
QIGN = 144.5 - (0.266 * TMPPRM) - (0.00058 * TMPPRM * TMPPRM) - (0.01 * TMPPRM * MC1) + 18.54 * (1.0 - exp(-0.151 * MC1)) + 6.4 * MC1

if (QIGN >= 344.0): 
    IC = 0
else:
    CHI = (344.0 - QIGN) / 10.0
    if ((pow(CHI, 3.66) * 0.000923 / 50.0) <= PNORM1):
        IC = 0
    else:
    
        PI = ((pow(CHI, 3.66) * 0.000923 / 50.0) - PNORM1) * 100.0 / PNORM2
        if (PI < 0.0): PI = 0.0
        if (PI > 100.0): PI = 100.0
        SCN = 100.0 * SC / SCM
        if (SCN > 100.0): SCN = 100.0
        PFI = pow(SCN, 0.5)
        IC = 0.10 * PI * PFI

if (SC < 0.00001): IC = 0.0


In [59]:
IC

25.02692142648017